In [ ]:
# Eseguire questo snippet per montare il drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Machine Learning Foundation

## Section 1, Part a: Reading Data 

### Obiettivi

 - Creare una connessione SQL ad un semplice database SQL, e leggere dei record da quel database.
 - Esplorare parametri di input comuni.


### Pacchetti

 - [Pandas](https://pandas.pydata.org/pandas-docs/stable/)
 - [Pandas.read_sql](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_sql.html)
 - [SQLite3](https://docs.python.org/3.6/library/sqlite3.html)

## Semplice lettura dei dati

Structured Query Language (SQL) è una [specifica ANSI](https://docs.oracle.com/database/121/SQLRF/ap_standard_sql001.htm#SQLRF55514), implementata da vari database. SQL è un formato potente per interagire con dei database molto grandi in modo efficiente. Inoltre SQL permette di avere una esperienza consistente tra un gran numero di database. Useremo sqlite, una versione leggera e ristretta di sql per questo esempio. sqlite usa una versione leggermente modificata di SQL, che potrebbe essere differente da quanto siamo abituati.



In [ ]:
# Imports
import sqlite3 as sq3
import pandas.io.sql as pds
import pandas as pd

### Connessioni ai database
Il nostro primo step è quello di creare una connessione al nostro database SQL. Alcuni dei database SQL comunemente usati in Python sono:
 - Microsoft SQL Server
 - Postgres
 - MySQL
 - AWS Redshift
 - AWS Aurora
 - Oracle DB
 - Terradata
 - Db2 Family
 - Many, many others
 
Ognuno di questi database richiederà un setup leggermente differente, e potrebbe richiedere delle credenziali (username & password), tokens, o altri tipi di requisiti di accesso. Useremo `sqlite3` per connetterci al nostro database, ma altri pacchetti per la connessione sono:

 - [`SQLAlchemy`](https://www.sqlalchemy.org/) (più comune)
 - [`psycopg2`](http://initd.org/psycopg/)
 - [`MySQLdb`](http://mysql-python.sourceforge.net/MySQLdb.html)

In [ ]:
# Inizializzazione del path per il database SQL
path = '/content/drive/MyDrive/Colab Notebooks/Coursera - Exploratory Data Analysis for Machine Learning/Coursera - Activity #1 - Reading Data/data/classic_rock.db'
con = sq3.Connection(path)

# Adesso abbiamo una connessione live al nostro database SQL

### Lettura dei dati

Adesso che abbiamo una connessione al database, possiamo performare delle quary, e caricare i risultati in DataFrames di pandas


In [ ]:
# Scrivi la query
query = '''
SELECT * 
FROM rock_songs;
'''

# Esegui la query
observations = pds.read_sql(query, con)

observations.head()

,Song,Artist,Release_Year,PlayCount
0,Caught Up in You,.38 Special,1982.0,82
1,Hold On Loosely,.38 Special,1981.0,85
2,Rockin' Into the Night,.38 Special,1980.0,18
3,Art For Arts Sake,10cc,1975.0,1
4,Kryptonite,3 Doors Down,2000.0,13


In [ ]:
# E' possibile eseguire qualsiasi query supportata da SQL
query = '''
SELECT Artist, Release_Year, COUNT(*) AS num_songs, AVG(PlayCount) AS avg_plays  
    FROM rock_songs
    GROUP BY Artist, Release_Year
    ORDER BY num_songs desc;
'''

# Esegui la query
observations = pds.read_sql(query, con)

observations.head()

,Artist,Release_Year,num_songs,avg_plays
0,The Beatles,1967.0,23,6.565217
1,Led Zeppelin,1969.0,18,21.000000
2,The Beatles,1965.0,15,3.800000
3,The Beatles,1968.0,13,13.000000
4,The Beatles,1969.0,13,15.000000


## Parametri comuni

Ci sono un po di parametri comuni che possono essere usati per leggere i dati SQL con della formattazione:

- coerce_float: Tenta di forzare i numeri in floats
- parse_dates: Lista di colonne da parsare a date
- chunksize: Numero di righe da includere in ogni chunk
 
 Diamo un'occhiata usando alcuni di questi parametri
 

In [ ]:
query='''
SELECT Artist, Release_Year, COUNT(*) AS num_songs, AVG(PlayCount) AS avg_plays  
    FROM rock_songs
    GROUP BY Artist, Release_Year
    ORDER BY num_songs desc;
'''

# Esegui la query
observations_generator = pds.read_sql(query,
                            con,
                            coerce_float=True, # Non ha effetto su questo data set, perché i float erano gia stati correttamente parsati
                            parse_dates=['Release_Year'], # Parsa `Release_Year`come una data 
                            chunksize=5 # Permette di streammare i risultati come una serie di tabelle più corte
                           )

for index, observations in enumerate(observations_generator):
    if index < 5:
        print(f'Observations index: {index}'.format(index))
        display(observations)

Observations index: 0


,Artist,Release_Year,num_songs,avg_plays
0,The Beatles,1970-01-01 00:32:47,23,6.565217
1,Led Zeppelin,1970-01-01 00:32:49,18,21.000000
2,The Beatles,1970-01-01 00:32:45,15,3.800000
3,The Beatles,1970-01-01 00:32:48,13,13.000000
4,The Beatles,1970-01-01 00:32:49,13,15.000000


Observations index: 1


,Artist,Release_Year,num_songs,avg_plays
0,Led Zeppelin,1970-01-01 00:32:50,12,13.166667
1,Led Zeppelin,1970-01-01 00:32:55,12,14.166667
2,Pink Floyd,1970-01-01 00:32:59,11,41.454545
3,Pink Floyd,1970-01-01 00:32:53,10,29.100000
4,The Doors,1970-01-01 00:32:47,10,28.900000


Observations index: 2


,Artist,Release_Year,num_songs,avg_plays
0,Fleetwood Mac,1970-01-01 00:32:57,9,35.666667
1,Jimi Hendrix,1970-01-01 00:32:47,9,24.888889
2,The Beatles,1970-01-01 00:32:43,9,2.444444
3,The Beatles,1970-01-01 00:32:44,9,3.111111
4,Elton John,1970-01-01 00:32:53,8,18.500000


Observations index: 3


,Artist,Release_Year,num_songs,avg_plays
0,Led Zeppelin,1970-01-01 00:32:51,8,47.750000
1,Led Zeppelin,1970-01-01 00:32:53,8,34.125000
2,Boston,1970-01-01 00:32:56,7,69.285714
3,Rolling Stones,1970-01-01 00:32:49,7,36.142857
4,Van Halen,1970-01-01 00:32:58,7,51.142857


Observations index: 4


,Artist,Release_Year,num_songs,avg_plays
0,Bruce Springsteen,1970-01-01 00:32:55,6,7.666667
1,Bruce Springsteen,1970-01-01 00:33:04,6,11.500000
2,Creedence Clearwater Revival,1970-01-01 00:32:49,6,23.833333
3,Creedence Clearwater Revival,1970-01-01 00:32:50,6,18.833333
4,Def Leppard,1970-01-01 00:33:07,6,32.000000


### Machine Learning Foundation (C) 2020 IBM Corporation